## CNN DEMO




In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.datasets import fetch_20newsgroups

## Load Dataset

In [5]:
# Load a binary classification dataset
categories = ['alt.atheism', 'sci.space']
print("Loading dataset...")


Loading dataset...


In [6]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    newsgroups.data, newsgroups.target,
    test_size=0.2, random_state=42
)

print(f"Training samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")

Training samples: 1428
Test samples: 358


In [7]:
# Better text preprocessing
print("Preprocessing text data...")
max_words = 10000  # Increased vocabulary size
max_length = 150   # Increased sequence length

# Tokenize the text
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

Preprocessing text data...


In [8]:
# Convert text to sequences
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequences
X_train_pad = pad_sequences(X_train_seq, maxlen=max_length, padding='post', truncating='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_length, padding='post', truncating='post')


## Build a CNN model

In [9]:
# Build a CNN model with slightly more capacity
print("Building CNN model...")
embedding_dim = 100  # Increased embedding dimension

model = Sequential([
    Embedding(max_words, embedding_dim),
    Conv1D(128, 5, activation='relu'),  # More filters
    GlobalMaxPooling1D(),
    Dropout(0.4),  # Increased dropout for better regularization
    Dense(64, activation='relu'),  # Added a hidden layer
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

# Compile with reduced learning rate for better convergence
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)
model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Display model summary
model.summary()

Building CNN model...


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_1          │ ?                      │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [10]:
# Train the model for more epochs
print("Training model...")
model.fit(
    X_train_pad, y_train,
    epochs=10,  # Increased epochs for better training
    batch_size=32,  # Smaller batch size for better generalization
    verbose=1
)

Training model...
Epoch 1/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 4s 47ms/step - accuracy: 0.5085 - loss: 0.6917
Epoch 2/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 46ms/step - accuracy: 0.5676 - loss: 0.6723
Epoch 3/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 46ms/step - accuracy: 0.7046 - loss: 0.6108
Epoch 4/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 62ms/step - accuracy: 0.8884 - loss: 0.4719
Epoch 5/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 56ms/step - accuracy: 0.9406 - loss: 0.2962
Epoch 6/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 46ms/step - accuracy: 0.9750 - loss: 0.1449
Epoch 7/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 46ms/step - accuracy: 0.9768 - loss: 0.0804
Epoch 8/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 47ms/step - accuracy: 0.9907 - loss: 0.0505
Epoch 9/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 59ms/step - accuracy: 0.9837 - loss: 0.0417
Epoch 10/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 59ms/step - accuracy: 0.9905 - loss: 0.0264


In [11]:
# Evaluate and predict on test data
print("\n--- Test Data Evaluation ---")
loss, accuracy = model.evaluate(X_test_pad, y_test, verbose=0)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


--- Test Data Evaluation ---
Test Loss: 0.1679
Test Accuracy: 0.9302


In [12]:
# Make predictions on test data
y_pred_prob = model.predict(X_test_pad, verbose=0)
y_pred = (y_pred_prob > 0.5).astype(int).flatten()

# Print full evaluation metrics
print("\nConfusion Matrix:")
cm = confusion_matrix(y_test, y_pred)
print(cm)

print("\nClassification Report:")
report = classification_report(y_test, y_pred, target_names=categories)
print(report)


Confusion Matrix:
[[144  13]
 [ 12 189]]

Classification Report:
              precision    recall  f1-score   support

 alt.atheism       0.92      0.92      0.92       157
   sci.space       0.94      0.94      0.94       201

    accuracy                           0.93       358
   macro avg       0.93      0.93      0.93       358
weighted avg       0.93      0.93      0.93       358



In [13]:
# Display some sample predictions
print("\n--- Sample Test Predictions ---")
for i in range(5):
    # Get a sample from test data
    sample_text = X_test[i][:100] + "..."  # Truncate for display
    true_label = categories[y_test[i]]
    pred_label = categories[y_pred[i]]
    confidence = y_pred_prob[i][0] if y_pred[i] == 1 else 1 - y_pred_prob[i][0]

    print(f"Sample {i+1}:")
    print(f"Text: {sample_text}")
    print(f"True label: {true_label}")
    print(f"Predicted label: {pred_label} (confidence: {confidence:.2f})")
    print(f"Correct prediction: {y_test[i] == y_pred[i]}")
    print("-" * 50)


--- Sample Test Predictions ---
Sample 1:
Text: Piper lived in my town (Williamsport, PA) when he killed himself.  It
was in the early '60's.  He ha...
True label: sci.space
Predicted label: alt.atheism (confidence: 0.94)
Correct prediction: False
--------------------------------------------------
Sample 2:
Text: 

Uh, no. These burst detectors are just that, burst detectors.
They have no angular resolution.

No...
True label: sci.space
Predicted label: sci.space (confidence: 0.98)
Correct prediction: True
--------------------------------------------------
Sample 3:
Text: 

Well, chimps must have some system.  They live in social groups
as we do, so they must have some "...
True label: alt.atheism
Predicted label: alt.atheism (confidence: 0.95)
Correct prediction: True
--------------------------------------------------
Sample 4:
Text: How about transferring control to a non-profit organisation that is
able to accept donations to keep...
True label: sci.space
Predicted label: sci.space